# Обзор базы данных крупного сервиса для чтения книг по подписке

**Описание:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Заказчик решил быть на волне и купил крупный сервис для чтения книг по подписке. 

**Исходные материалы:**


В нашем распоряжении база данных куда входят:
- Таблица `books` — Содержит данные о книгах
- Таблица `authors` — Содержит данные об авторах
- Таблица `publishers` — Содержит данные об издательствах
- Таблица `ratings` — Содержит данные о пользовательских оценках книг
- Таблица `reviews` — Содержит данные о пользовательских обзорах

**Цели:**
 - Проанализировать базу данных.


**Ход выполнения:**
- Получим доступ к базе данных
- Ознакомимся с данными
- Исследуем базу данных и сделаем следующее:
    - Выясним сколько книг вышло после 1 января 2000 года
    - Для каждой книги посчитаем количество обзоров и среднюю оценку
    - Найдем самое крупное издательство по объему выпущенных книг толщиной более 50 страниц
    - Выясним какой автор имеют самую высокую среднюю оценку среди книг, у которых есть 50 оценок и более
    - Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок
- Сформируем выводы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

## Первый взгляд на данные

В первую очередь необходимо получить данные, для этого подключимся к базе данных:

In [2]:
# устанавливаем параметры соединения
db_config = {
    'user': '***',                                 # имя пользователя
    'pwd': '***',                                  # пароль
    'host': '***',
    'port': ***,                                   # порт подключения
    'db': '***'                                    # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query):
    con=engine.connect()
    return pd.io.sql.read_sql(sql=text(query), con = con)

In [5]:
# проверим корректность работы. Сформируем первый запрос
query = (
    '''SELECT *
       FROM books
       LIMIT 5'''
)

# выполняем запрос
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Доступ к базе данных получен. Посмотрим на первые строки таблиц и оценим их объем. Первые строки Таблицы `books` уже есть, посмотрим сколько всего в таблице данных:

In [6]:
# найдем количество строк в `books`
query = (
    '''SELECT COUNT(*)
       FROM books'''
)
get_sql_data(query)

,count
0,1000


В таблице `books` - 1 000 записей.

Изучим информацию о других таблицах

In [7]:
# функция, которая формирует SQL-запрос для вывода первых 5 строк таблицы
def get_table_head(table):
    return f'SELECT * FROM {table} LIMIT 5'

In [8]:
# посмотрим на первые строки 'authors'
get_sql_data(get_table_head('authors'))

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
# 'publishers'
get_sql_data(get_table_head('publishers'))

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
# 'ratings'
get_sql_data(get_table_head('ratings'))

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
# 'reviews'
get_sql_data(get_table_head('reviews'))

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Мы изучили первые строки каждой таблици, теперь изучим их объем.

In [12]:
# функция, которая формирует SQL-запрос для вывода количества строк в таблице
def get_table_rows(table):
    return f'SELECT COUNT(*) FROM {table}'

In [13]:
# посмотрим на объем 'authors'
get_sql_data(get_table_rows('authors'))

,count
0,636


In [14]:
# 'publishers'
get_sql_data(get_table_rows('publishers'))

,count
0,340


In [15]:
# 'ratings'
get_sql_data(get_table_rows('ratings'))

,count
0,6456


In [16]:
# 'reviews'
get_sql_data(get_table_rows('reviews'))

,count
0,2793


### Результаты обзора данных

В нашем распоряжении база данных куда входят:


- Таблица **`books`** — Содержит 1000 записей о книгах:
    - `book_id` — идентификатор книги
    - `author_id` — идентификатор автора
    - `title` — название книги
    - `num_pages` — количество страниц
    - `publication_date` — дата публикации книги
    - `publisher_id` — идентификатор издателя
    
    
- Таблица **`authors`** — Содержит 636 записей об авторах книг:
    - `author_id` — идентификатор автора
    - `author` — имя автора
    
    
- Таблица **`publishers`** — Содержит 340 записей об издательствах:
    - `publisher_id` — идентификатор издательства
    - `publisher` — название издательства
    
    
- Таблица **`ratings`** — Содержит 6456 записей о пользовательских оценках книг:
    - `rating_id` — идентификатор оценки
    - `book_id` — идентификатор книги
    - `username` — имя пользователя, оставившего оценку
    - `rating` — оценка книги


- Таблица **`reviews`** — Содержит 2793 записи о пользовательских обзорах:
    - `review_id` — идентификатор обзора
    - `book_id` — идентификатор книги
    - `username` — имя автора обзора
    - `text` — текст обзора
    
    
Ожидаемо меньше всего данных с издательствами(340 записей), а больше всего данных о пользовательских активностях: оценки (6 456 записей) и обзоры (2 393 записи)

## Исследование базы данных
### Cколько книг вышло после 1 января 2000 года?

In [17]:
# в теории одна и таже книга может выходить несколько раз разными изданиями
# поэтому найдем общее количество уникальных названий книг дата издания которых позже 2000-01-01
query_task_1 = (
    '''SELECT COUNT(DISTINCT title)
       FROM books 
       WHERE CAST(publication_date AS DATE) > '2000-01-01';'''
)

# выполняем запрос
get_sql_data(query_task_1)

,count
0,818


In [18]:
# проверим есть ли книги с одинаковыми названиями
query_task_1_1 = (
    '''SELECT COUNT(title)
       FROM books 
       WHERE CAST(publication_date AS DATE) > '2000-01-01';'''
)

# выполняем запрос
get_sql_data(query_task_1_1)

,count
0,819


Действительно, есть 2 книги с одинаковыми названиями, посмотрим это одна и таже книга разных лет или просто совпадение:

In [19]:
# проверим есть ли книги с одинаковыми названиями
query_task_1_1_1 = (
    '''SELECT *
       FROM books
       WHERE title IN (SELECT title
                       FROM books 
                       WHERE CAST(publication_date AS DATE) > '2000-01-01'
                       GROUP BY title
                       ORDER BY COUNT(title) DESC
                       LIMIT 1);'''
)

# выполняем запрос
get_sql_data(query_task_1_1_1)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


In [20]:
# посмотрим на автора
author = (
    '''SELECT author
       FROM authors
       WHERE author_id = 39'''
)

# выполняем запрос
get_sql_data(author)

,author
0,Arthur Golden


С 2000 года вышли 2 книги "Мемуары Гейши" Артура Голдена в разных изданиях, при чем впервые этот роман увидел свет в 1997 году. 

После 1 января 2000 года вышло 819 книг, но книга "Мемуары Гейши" выходила дважды.

### Количество обзоров и средняя оценка книг

In [21]:
# сначала найдем общее количество обзоров
# далее для каждой книги найдем средний рейтинг
# полученные данные присоединим к основной таблице с книгами с помощью LEFT JOIN
# так в итогой таблице будут все исходные книги, не зависимо от того есть ли у этой книги оценки или обзоры
query_task_2 = (
    '''WITH rev AS (SELECT book_id,
                           COUNT(review_id) AS total_reviews
                    FROM reviews
                    GROUP BY book_id),
            rat AS (SELECT book_id,
                           ROUND(AVG(rating), 2) AS avg_rating
                    FROM ratings
                    GROUP BY book_id)
    
       SELECT b.title,'''
              'CAST(rev.total_reviews AS int4),' # не понимаю почему остается десятичный знак а не целое число((
              '''rat.avg_rating
       FROM books AS b 
       LEFT JOIN rev ON b.book_id=rev.book_id
       LEFT JOIN rat ON b.book_id=rat.book_id;'''
)

# выполняем запрос
get_sql_data(query_task_2)

,title,total_reviews,avg_rating
0,The Body in the Library (Miss Marple #3),2.0,4.50
1,Galápagos,2.0,4.50
2,A Tree Grows in Brooklyn,5.0,4.25
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.00
4,The Prophet,4.0,4.29
...,...,...,...
995,Alice in Wonderland,4.0,4.23
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.00
997,Christine,3.0,3.43
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.50


Таблица с количеством обзоров и средней оценкой для каждой книги готова

### Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [22]:
# получим идентификатор издателя, который выпустил больше всех книг толще 50 страниц
# и найдем наименование издательства по его идентификатору
query_task_3 = (
    '''SELECT publisher
       FROM publishers
       WHERE publisher_id = (
                               SELECT publisher_id                                      
                               FROM books
                               WHERE num_pages > 50
                               GROUP BY publisher_id
                               ORDER BY COUNT(book_id) DESC
                               LIMIT 1
                            )'''
)

# выполняем запрос
get_sql_data(query_task_3)

,publisher
0,Penguin Books


In [23]:
# проверим какое издательство выпустило больше всего уникальных книг толще 50 страниц
query_task_3_1 = (
    '''SELECT publisher
       FROM publishers
       WHERE publisher_id = (
                               SELECT publisher_id
                               FROM books
                               WHERE num_pages > 50
                               GROUP BY publisher_id
                               ORDER BY COUNT(DISTINCT book_id) DESC
                               LIMIT 1
                            )'''
)

# выполняем запрос
get_sql_data(query_task_3_1)

,publisher
0,Penguin Books


Издательство, выпустившее наибольшее число книг толще 50 страниц - "Penguin Books"

### Какой автор имеют самую высокую среднюю оценку среди книг, у которых есть 50 и более оценок?

In [24]:
# сначала найдем все книги, у которых есть 50 оценок и более
# далее для каждой книги посчитаем среднюю оценку и объединим данные
# сгруппируем полученные данные по id авторов и найдем среднюю оценку их книг, 
# отсортируем и оставим пятерых у кого средняя оценка книг выше всех
query_task_4 = (
    '''WITH b AS (SELECT *
                  FROM books
                  WHERE book_id IN (SELECT book_id
                                    FROM ratings
                                    GROUP BY book_id
                                    HAVING COUNT(rating_id) >= 50)),
                                    
            r AS (SELECT book_id,
                         ROUND(AVG(rating), 2) AS avg_rating
                  FROM ratings
                  GROUP BY book_id),

            top_author AS (SELECT author_id,
                                  AVG(avg_rating) AS avg_rating
                                  FROM b LEFT JOIN r ON b.book_id=r.book_id
                                  GROUP BY author_id
                                  ORDER BY AVG(avg_rating) DESC
                                  LIMIT 5)
                                  
       SELECT a.author,
              tp.avg_rating
       FROM authors AS a INNER JOIN top_author AS tp ON a.author_id=tp.author_id
       '''
)

# выполняем запрос
get_sql_data(query_task_4)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.285
1,Markus Zusak/Cao Xuân Việt Khương,4.260
2,J.R.R. Tolkien,4.260
3,Louisa May Alcott,4.190
4,Rick Riordan,4.080


В Топ-5 авторов с самыми высокооцененными работами вошли: Рик Риордан, Луиза Мэй Олкотт, Дж.Р.Р. Толкин и Маркус Зузак.
А первое место со средней оценкой книг 4.285 достается Джоан Роулинг.

### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [25]:
# сначала найдем пользователей, которые поставили более 48 оценок
# потом для для каждого пользователя который входит в число первых найдем количество обзоров
# после чего найдем среднее количество обзоров среди этих пользователей
query_task_5 = (
        '''SELECT AVG(total_reviews)
           FROM (SELECT username,
                        COUNT(review_id) AS total_reviews
                 FROM reviews
                 WHERE username IN (SELECT DISTINCT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating_id) > 48)
                                    GROUP BY username) AS top_rew'''
)

# выполняем запрос
get_sql_data(query_task_5)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24 обзора

## Выводы:


- Доступ к базе данных настроен
- Приведено описание данных во всех таблицах
    - Таблица **`books`** — Содержит 1000 записей о книгах
    - Таблица **`authors`** — Содержит 636 записей об авторах книг
    - Таблица **`publishers`** — Содержит 340 записей об издательствах
    - Таблица **`ratings`** — Содержит 6456 записей о пользовательских оценках книг
    - Таблица **`reviews`** — Содержит 2793 записи о пользовательских обзорах

**Исследование базы данных:**

- После 1 января 2000 года вышло 819 книг, но книга "Мемуары Гейши" выходила дважды
- Таблица с количеством обзоров и средней оценкой для каждой книги сформирована
- Издательство, выпустившее наибольшее число книг толще 50 страниц - "Penguin Books"
- Джоан Роулинг имеет самую высокую среднюю оценку среди книг, у которых есть более 50 оценок (**4.285**)
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24 обзора